In [19]:
from basketball_reference_web_scraper import client
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
import lxml.html as lh
import numpy as np
import janitor
import unidecode


In [2]:
advanced_stats = client.players_advanced_season_totals(season_end_year=2019)

normal_stats = client.players_season_totals(season_end_year=2018)

testdf = pd.DataFrame(advanced_stats)

In [6]:
testdf['minutes_played_total'] = testdf.groupby('name').minutes_played.transform('sum')
testdf['proportion'] = testdf['minutes_played']/testdf['minutes_played_total']
num_cols = list(testdf.select_dtypes(include=['int', 'float64']))
unwanted_num_cols = ['age', 'minutes_played', 'games_played']
for col in unwanted_num_cols:
    num_cols.remove(col)
for col in num_cols:
    testdf[col] = testdf[col]*testdf['proportion']
testdf = testdf.groupby('name').agg('sum')

player_counts = pd.DataFrame(testdf.groupby('name').size())

In [31]:
def clean_advanced(year):
    advanced_stats = client.players_advanced_season_totals(season_end_year=year)
    df = pd.DataFrame(advanced_stats)
    df['minutes_played_total'] = df.groupby('name').minutes_played.transform('sum')
    df['proportion'] = df['minutes_played']/df['minutes_played_total']
    num_cols = list(df.select_dtypes(include=['int', 'float64']))
    unwanted_num_cols = ['age', 'minutes_played', 'games_played']
    for col in unwanted_num_cols:
        num_cols.remove(col)
    for col in num_cols:
        df[col] = df[col]*df['proportion']
    df = df.groupby('name').agg('sum')
    df['year'] = year
    return df

In [36]:
a_stats_2011_2020 = []
for year in range(2011, 2020):
    df = clean_advanced(year)
    a_stats_2011_2020.append(df)
a_stats_2011_2020 = pd.concat(a_stats_2011_2020)
a_stats_2011_2020 = a_stats_2011_2020.reset_index()





In [35]:
for i in range(len(a_stats_2011_2020)):
    a_stats_2011_2020['name'][i] = unidecode.unidecode(a_stats_2011_2020['name'][i])


KeyError: 'name'